In [1]:
import pandas
import matplotlib.pyplot as plt
pandas.set_option('display.float_format', lambda x: '%.4f' % x)
import numpy as np
import epydemiology as epy

Error: PyMSSQL package not available. Some features may not be available.


## Revised definition of rejected (Aug. 4)
FOR ALL DATA:
- Remove one-stop data
- ballot_req_type == "MAIL"
- valid return date only 

In [2]:
df = pandas.read_csv('../data/absentee_20181106_w_ethnicity.csv', encoding='latin1')
len(df)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2111797

In [3]:
# remove one-stop
df = df[df['ballot_req_type'] == "MAIL"]
len(df)

137234

In [4]:
# valid return dt only
df = df[df['ballot_rtn_dt'].notnull()]
len(df)

104091

In [5]:
# Vote breakdown
df['ballot_rtn_status'].value_counts(normalize=True, dropna=False) *100

ACCEPTED                  93.7363
VOTER SIGNATURE MISSING    2.7140
WITNESS INFO INCOMPLETE    1.6121
RETURNED AFTER DEADLINE    0.8435
SPOILED                    0.7407
NOT PROPERLY NOTARIZED     0.1528
RETURNED UNDELIVERABLE     0.1287
SIGNATURE DIFFERENT        0.0317
PENDING                    0.0259
DUPLICATE                  0.0096
CONFLICT                   0.0048
Name: ballot_rtn_status, dtype: float64

In [6]:
df['ballot_rtn_status'].unique()

array(['ACCEPTED', 'SPOILED', 'PENDING', 'WITNESS INFO INCOMPLETE',
       'VOTER SIGNATURE MISSING', 'RETURNED UNDELIVERABLE',
       'RETURNED AFTER DEADLINE', 'CONFLICT', 'SIGNATURE DIFFERENT',
       'NOT PROPERLY NOTARIZED', 'DUPLICATE'], dtype=object)

 ## Race risk ratios  

In [7]:
# Calc risk ratios, compared to white group, for each category
RACES = df.race.unique()

for R in RACES: 
    #print(R)
    if(R == 'WHITE'):
        continue
    else:
        df_temp = df[df['race'].isin([R, 'WHITE'])]
        df_temp['ballot_accepted'] = df_temp['ballot_rtn_status'].apply(lambda x: True if (x == "ACCEPTED") else False)
        print(epy.phjRelativeRisk(df_temp, 'race', R, 'ballot_accepted', True).to_string())
        print()
        print()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


                 UNDESIGNATED  WHITE  total   risk     rr 95CI_katz_llim 95CI_katz_ulim
ballot_accepted                                                                        
False                     281   3773   4054 0.0693 1.1594         1.0323         1.3022
True                     4730  74388  79118 0.0598 1.0000            ---            ---


                 ASIAN  WHITE  total   risk     rr 95CI_katz_llim 95CI_katz_ulim
ballot_accepted                                                                 
False              178   3773   3951 0.0451 1.4785         1.2738         1.7160
True              2338  74388  76726 0.0305 1.0000            ---            ---


                 BLACK or AFRICAN AMERICAN  WHITE  total   risk     rr 95CI_katz_llim 95CI_katz_ulim
ballot_accepted                                                                                     
False                                 2022   3773   5795 0.3489 2.3621         2.2726         2.4551
True             

## Let's also do overall minority vs white voters. Drop undesignated

In [27]:
df['is_minority'] = df['race'].apply(lambda x: "NOT_MINORITY"if x=='WHITE' else "IS_MINORITY")

# let's skip undesignated
df_temp = df[df['race']!="UNDESIGNATED"]
df_temp[['race','is_minority']].head()

,race,is_minority
3,WHITE,NOT_MINORITY
49,ASIAN,IS_MINORITY
78,WHITE,NOT_MINORITY
79,WHITE,NOT_MINORITY
83,WHITE,NOT_MINORITY


## What percent minority, excluding undesignated?

In [49]:
df_temp.is_minority.value_counts(normalize=True)

NOT_MINORITY   0.7889
IS_MINORITY    0.2111
Name: is_minority, dtype: float64

In [24]:
df_temp.is_minority.unique()

array(['NOT_MINORITY', 'IS_MINORITY'], dtype=object)

In [25]:
df_temp['ballot_accepted'] = df_temp['ballot_rtn_status'].apply(lambda x: True if (x == "ACCEPTED") else False)
print(epy.phjRelativeRisk(df_temp, 'is_minority', "IS_MINORITY", 'ballot_accepted', True).to_string())
print()
print()

                 IS_MINORITY  NOT_MINORITY  total   risk     rr 95CI_katz_llim 95CI_katz_ulim
ballot_accepted                                                                              
False                   2466          3773   6239 0.3953 1.9886         1.9235         2.0560
True                   18453         74388  92841 0.1988 1.0000            ---            ---




/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
len(df[df['race'] == 'BLACK or AFRICAN AMERICAN'])

14915

In [32]:
len(df)

104091

In [33]:
14915 / 104091

0.14328808446455504

In [35]:
df['ballot_request_party'].value_counts()

DEM    43631
UNA    31818
REP    28252
LIB      367
GRE       17
CST        6
Name: ballot_request_party, dtype: int64

In [38]:
df[df['ballot_request_party'] == "DEM"]['race'].value_counts(normalize=True)

WHITE                              0.6157
BLACK or AFRICAN AMERICAN          0.2852
UNDESIGNATED                       0.0447
OTHER                              0.0204
ASIAN                              0.0194
TWO or MORE RACES                  0.0081
INDIAN AMERICAN or ALASKA NATIVE   0.0065
Name: race, dtype: float64

In [39]:
df[df['ballot_request_party'] == "REP"]['race'].value_counts(normalize=True)

WHITE                              0.9317
UNDESIGNATED                       0.0245
ASIAN                              0.0138
OTHER                              0.0133
BLACK or AFRICAN AMERICAN          0.0099
INDIAN AMERICAN or ALASKA NATIVE   0.0042
TWO or MORE RACES                  0.0026
Name: race, dtype: float64

In [45]:
100- (df['ballot_rtn_status'].value_counts(normalize=True)) * 100

ACCEPTED                   6.2637
VOTER SIGNATURE MISSING   97.2860
WITNESS INFO INCOMPLETE   98.3879
RETURNED AFTER DEADLINE   99.1565
SPOILED                   99.2593
NOT PROPERLY NOTARIZED    99.8472
RETURNED UNDELIVERABLE    99.8713
SIGNATURE DIFFERENT       99.9683
PENDING                   99.9741
DUPLICATE                 99.9904
CONFLICT                  99.9952
Name: ballot_rtn_status, dtype: float64

In [46]:
100- (df[df['race']=='BLACK or AFRICAN AMERICAN']['ballot_rtn_status'].value_counts(normalize=True)) * 100

ACCEPTED                  13.5568
VOTER SIGNATURE MISSING   94.5089
WITNESS INFO INCOMPLETE   95.0721
SPOILED                   98.4244
RETURNED AFTER DEADLINE   98.9407
RETURNED UNDELIVERABLE    99.7989
NOT PROPERLY NOTARIZED    99.8123
SIGNATURE DIFFERENT       99.9330
DUPLICATE                 99.9799
CONFLICT                  99.9799
PENDING                   99.9933
Name: ballot_rtn_status, dtype: float64

In [47]:
100- (df[df['race']=='WHITE']['ballot_rtn_status'].value_counts(normalize=True)) * 100

ACCEPTED                   4.8272
VOTER SIGNATURE MISSING   97.8020
WITNESS INFO INCOMPLETE   99.0289
RETURNED AFTER DEADLINE   99.2515
SPOILED                   99.4179
NOT PROPERLY NOTARIZED    99.8503
RETURNED UNDELIVERABLE    99.8836
PENDING                   99.9719
SIGNATURE DIFFERENT       99.9757
DUPLICATE                 99.9923
CONFLICT                  99.9987
Name: ballot_rtn_status, dtype: float64

In [51]:
df['ballot_rtn_dt'] = pandas.to_datetime(df.ballot_rtn_dt)